In [74]:
import json
import sys 
sys.path.append("/home/nbuser/library")
import time
import pandas as pd
import requests 


In [75]:
def get_latlng(address):
    key = {"key" : "40c1ae6866a363"}
    url = "https://us1.locationiq.com/v1/search.php"
    params = {
        'key' : key["key"],
        'q' : address,
        'format' : 'json'
    }
    time.sleep(0.2)
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
            return pd.Series({'Latitude' : response.status_code,  'Longitude' : response.status_code})
    
    lat = response.json()[0]['lat']
    lng = response.json()[0]['lon']
    return pd.Series({'Latitude': lat, 'Longitude':lng})

In [76]:
get_latlng(None)

Latitude     400
Longitude    400
dtype: int64

### Finding Lat and Lngfor every property in the file 

In [77]:
df = pd.read_csv("property.csv")
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park


In [78]:
df = df.drop(columns=['Latitude', 'Longitude', 'SA1'])

In [86]:
df[['Latitude', 'Longitude']] = df.Address.apply(get_latlng)

In [87]:
df.tail()

,Bedrooms,Bathrooms,Address,Land area,CV,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,Latitude,Longitude
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,48,27,24,21,24,21,Manurewa,-37.012914,174.903973
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,42,18,12,21,15,30,Karaka,-37.0636507,174.9228185
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,42,18,12,21,15,30,Karaka,-37.0636215,174.9239312
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,42,6,21,21,12,15,Onehunga,-36.91325,174.7874144
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,93,27,33,30,21,33,Clover Park,-36.9789324,174.8926273


In [89]:
def combine_lat_lng(lat, lng):
    return (lat,lng)

In [90]:
df['Combined coord'] = df.apply(lambda row : combine_lat_lng(row['Latitude'], row['Longitude']), axis=1)

In [91]:
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,Latitude,Longitude,Combined coord
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,48,27,24,21,24,21,Manurewa,-37.012914,174.903973,"(-37.012914, 174.903973)"
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,42,18,12,21,15,30,Karaka,-37.0636507,174.9228185,"(-37.0636507, 174.9228185)"
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,42,18,12,21,15,30,Karaka,-37.0636215,174.9239312,"(-37.0636215, 174.9239312)"
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,42,6,21,21,12,15,Onehunga,-36.91325,174.7874144,"(-36.91325, 174.7874144)"
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,93,27,33,30,21,33,Clover Park,-36.9789324,174.8926273,"(-36.9789324, 174.8926273)"
